In [1]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# CARGAMOS DATOS
train = pd.read_csv('../data/train.csv')
train.head()

,book_title,rating,reviewer,reviewer_rating,genre_ Action,genre_ Activities,genre_ Adult,genre_ Adventure,genre_ Alphabet,genre_ Animals,...,word_desc_two,word_desc_want,word_desc_way,word_desc_well,word_desc_work,word_desc_world,word_desc_would,word_desc_writing,word_desc_year,word_desc_years
0,The Woman in Me,4.5,Murderess Marbie,4,False,False,False,False,False,False,...,0.000000,0.000000,0.299337,0.000000,0.092395,0.000000,0.291034,0.268176,0.0,0.000000
1,The Woman in Me,4.5,L J,5,False,False,False,False,False,False,...,0.000000,0.000000,0.127401,0.000000,0.000000,0.000000,0.123868,0.000000,0.0,0.000000
2,The Woman in Me,4.5,Jamie,5,False,False,False,False,False,False,...,0.081837,0.240994,0.000000,0.067106,0.083909,0.077308,0.132151,0.000000,0.0,0.080612
3,The Woman in Me,4.5,KMG,5,False,False,False,False,False,False,...,0.093898,0.092171,0.233930,0.000000,0.000000,0.000000,0.227441,0.093146,0.0,0.277478
4,The Woman in Me,4.5,Stephanie Brown,5,False,False,False,False,False,False,...,0.000000,0.000000,0.000000,0.251972,0.000000,0.290281,0.000000,0.000000,0.0,0.000000


In [3]:
# MATRIZ USUARIO_LIBRO
matriz_usuario_libro = train.pivot_table(index='reviewer', columns='book_title', values='reviewer_rating')

# DICCIONARIO CON PROMEDIO DE CALIFICACIÓN
promedio_calificacion_libro = train.set_index('book_title')['rating'].to_dict()

# LLENAR VALORES FALTANTES DE MATRIZ USUARIO_LIBRO CON PROMEDIO
matriz_usuario_libro_rellena = matriz_usuario_libro.apply(lambda col: col.fillna(promedio_calificacion_libro.get(col.name, 0)))

In [4]:
# APLICAR SVD
U, sigma, Vt = svds(matriz_usuario_libro_rellena.values, k=50)
sigma = np.diag(sigma)

# RECONSTRUIR MATRIZ DE CALIFICACIONES
matriz_calificaciones_predicha = np.dot(np.dot(U, sigma), Vt)
calificaciones_predichas_df = pd.DataFrame(matriz_calificaciones_predicha, 
                                           index=matriz_usuario_libro_rellena.index, 
                                           columns=matriz_usuario_libro_rellena.columns)

# ESCALAR PREDICCIONES
calificaciones_predichas_df = calificaciones_predichas_df.clip(0, 5).round(1)

calificaciones_predichas_df.head()

book_title,"A Court of Mist and Fury (A Court of Thorns and Roses, 2)","A Court of Thorns and Roses (A Court of Thorns and Roses, 1)",A Court of Thorns and Roses Paperback Box Set (5 books),"A Court of Wings and Ruin (A Court of Thorns and Roses, 3)",A Little Life,All the Light We Cannot See: A Novel,Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones,"Brown Bear, Brown Bear, What Do You See?",Chicka Chicka Boom Boom (Board Book),Demon Copperhead: A Pulitzer Prize Winner,...,The Wonderful Things You Will Be,The Wonky Donkey,There Was an Old Lady Who Swallowed a Turkey!,Things We Never Got Over (Knockemout),Tom Lake: A Reese's Book Club Pick,Turkey Trouble,Verity,Where's Bluey?: A Search-and-Find Book,Where's Spot?,"World of Eric Carle, Around the Farm 30-Button Animal Sound Book - Great for First Words - PI Kids"
reviewer,,,,,,,,,,,,,,,,,,,,,
*AT,4.8,4.6,4.8,4.8,4.5,4.5,4.8,4.9,4.9,4.6,...,4.9,4.8,4.8,4.5,4.3,4.8,4.6,4.8,4.9,4.8
A H Kobayashi,4.8,4.6,4.8,4.8,4.5,4.5,4.8,4.9,4.9,4.6,...,4.9,4.8,4.8,4.5,4.3,4.8,4.6,4.8,4.9,4.8
A Reviewer,4.8,4.6,4.8,4.8,4.5,4.5,4.8,4.9,4.9,4.6,...,4.9,4.8,4.8,4.5,4.3,4.8,4.6,4.8,4.9,4.8
A. K. P.,4.8,4.6,4.8,4.8,4.5,4.5,4.8,4.9,4.9,4.6,...,4.9,4.8,4.8,4.5,4.3,4.8,4.6,4.8,4.9,4.8
A. Slater,4.8,4.6,4.8,4.8,4.5,4.5,4.8,4.9,4.9,4.6,...,4.9,4.8,4.8,4.5,4.3,4.8,4.6,4.8,4.9,4.8


In [5]:
# SIMILITUD COSENO ENTRE LIBROS
# Seleccionar las columnas que representan géneros y palabras clave de las reseñas
columnas_genero = [col for col in train.columns if col.startswith('genre_')]
columnas_palabras = [col for col in train.columns if col.startswith('word_desc_')]
datos_similitud = train[columnas_genero + columnas_palabras].copy()

# COLUMNAS BOOLEANAS A ENTEROS
columnas_booleanas = datos_similitud.select_dtypes(include=['bool']).columns.tolist()
datos_similitud[columnas_booleanas] = datos_similitud[columnas_booleanas].astype(int)

# CALCULAR SIMILITUD DE COSENO ENTRE LIBROS
similitud_coseno = cosine_similarity(datos_similitud)

print(similitud_coseno)

[[1.         0.66173395 0.64893982 ... 0.14376725 0.07734853 0.07734853]
 [0.66173395 1.         0.65182141 ... 0.01621683 0.04272846 0.04272846]
 [0.64893982 0.65182141 1.         ... 0.01426251 0.03368796 0.03368796]
 ...
 [0.14376725 0.01621683 0.01426251 ... 1.         0.20383083 0.70383083]
 [0.07734853 0.04272846 0.03368796 ... 0.20383083 1.         0.5       ]
 [0.07734853 0.04272846 0.03368796 ... 0.70383083 0.5        1.        ]]


In [6]:
# FUNCIÓN PARA OBTENER RECOMENDACIONES HÍBRIDAS CON SVD Y COSENO DE SIMILITUD
def recomendaciones_hibridas(usuario_id, num_recomendaciones=5, peso_similitud=0.7):
    # Obtener libros que el usuario ha calificado alto
    libros_calificados_usuario = matriz_usuario_libro.loc[usuario_id].dropna()
    libros_bien_calificados = libros_calificados_usuario[libros_calificados_usuario >= 4].index.tolist()
    
    # Obtener las predicciones de calificación ordenadas para ese usuario
    predicciones_usuario = calificaciones_predichas_df.loc[usuario_id].sort_values(ascending=False)

    recomendaciones = []
    
    for titulo_libro, calificacion_predicha in predicciones_usuario.items():
        if titulo_libro not in libros_calificados_usuario.index:
            # Obtener el índice del libro en train_df
            indice_libro = train[train['book_title'] == titulo_libro].index[0]
            
            # Calcular la similitud entre el libro y los libros de alta calificación del usuario
            indices_libros_bien_calificados = [train[train['book_title'] == titulo].index[0] for titulo in libros_bien_calificados]
            if indices_libros_bien_calificados:
                puntaje_similitud = similitud_coseno[indice_libro, indices_libros_bien_calificados].mean()
            else:
                puntaje_similitud = similitud_coseno[indice_libro].mean()
            
            # Normalizar calificacion_predicha y puntaje_similitud a un rango de 0 a 1
            calificacion_predicha_norm = (calificacion_predicha - calificaciones_predichas_df.values.min()) / (calificaciones_predichas_df.values.max() - calificaciones_predichas_df.values.min())
            puntaje_similitud_norm = (puntaje_similitud - similitud_coseno.min()) / (similitud_coseno.max() - similitud_coseno.min())
            
            # Calcular un puntaje ponderado combinado (calificacion_predicha y puntaje_similitud)
            puntaje_combinado = (peso_similitud * puntaje_similitud_norm) + ((1 - peso_similitud) * calificacion_predicha_norm)
            
            recomendaciones.append((titulo_libro, calificacion_predicha, puntaje_similitud, puntaje_combinado))
    
    # Ordenar recomendaciones por puntaje combinado
    recomendaciones = sorted(recomendaciones, key=lambda x: x[3], reverse=True)
    recomendaciones_top = recomendaciones[:num_recomendaciones]
    
    # Retorna los libros recomendados con título, calificación predicha, puntaje de similitud y puntaje combinado
    resultado_df = pd.DataFrame(recomendaciones_top, columns=['Título del Libro', 'Calificación Predicha', 'Puntaje de Similitud', 'Puntaje Combinado'])
    
    # Asegurarnos de que los títulos de libros se vean completamente y claramente
    pd.set_option('display.max_colwidth', None)
    
    return resultado_df

In [7]:
# Ejemplo de uso con mayor peso en la similitud de libros
usuario_ejemplo = "A H Kobayashi"
recomendaciones_usuario = recomendaciones_hibridas(usuario_ejemplo, peso_similitud=0.8)
print(f"Recomendaciones híbridas para {usuario_ejemplo}:\n", recomendaciones_usuario)

Recomendaciones híbridas para A H Kobayashi:
                                Título del Libro  Calificación Predicha  \
0    Harry Potter Paperback Box Set (Books 1-7)                    4.9   
1        Don't Let the Pigeon Drive the Sleigh!                    4.8   
2                                Goodnight Moon                    4.9   
3  Taylor Swift: A Little Golden Book Biography                    4.9   
4                   The Very Hungry Caterpillar                    4.9   

   Puntaje de Similitud  Puntaje Combinado  
0              0.140692           0.305887  
1              0.131188           0.291617  
2              0.116369           0.286429  
3              0.114947           0.285291  
4              0.112042           0.282967  
